In [1]:
import os
import pandas as pd
import torch
import gc
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
from PerplexityEvaluator import PerplexityEvaluator
from PrepareSentenceContext import PrepareSentenceContext

In [2]:
# !pip install --break-system-packages datasets  
# !pip install --break-system-packages ipywidgets
# !pip install --break-system-packages --upgrade transformers "numpy<2" "pyarrow<14"

In [3]:
torch.cuda.empty_cache()
gc.collect()
!nvidia-smi

Tue Mar  4 08:18:08 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100 80GB PCIe          Off |   00000001:00:00.0 Off |                    0 |
| N/A   29C    P0             44W /  300W |       6MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [4]:
def generate_responses(
    input_path, model_name, context_policy, author,
    output_dir="Responses", target_rows=100, checkpoint_path=None
):
    """
    Generate responses for a dataset row by row, ensuring that the target number of rows from the dataset is processed.

    Parameters:
    - input_path: Path to the input dataset CSV.
    - model_name: Name of the language model for perplexity evaluation.
    - context_policy: Context policy to use for sentence processing.
    - author_column: Column containing the text to process.
    - output_dir: Directory where responses will be saved.
    - target_rows: Number of rows from the dataset to process in this run.
    - checkpoint_path: Path to save progress for crash recovery.
    """
    # Load input dataset
    df = pd.read_csv(input_path)
    dataset_name = os.path.basename(input_path).split("_")[0]

    # Prepare output file path
    if "/" in model_name:
        lm_name_str = model_name.split("/")[-1]
    else:
        lm_name_str = model_name
    save_path = os.path.join(
        output_dir, f"{dataset_name}_{author}_{context_policy}_{lm_name_str}.csv"
    )
    os.makedirs(output_dir, exist_ok=True)

    # Load existing responses if file exists
    if os.path.exists(save_path):
        responses_df = pd.read_csv(save_path)
    else:
        responses_df = pd.DataFrame(columns=["num", "length", "response", "context_length", "name"])

    # Initialize model, tokenizer, and utilities
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)
    model.eval()

    evaluator = PerplexityEvaluator(model, tokenizer)
    parser = PrepareSentenceContext(context_policy=context_policy)

    # Process rows individually
    with tqdm(total=target_rows, desc="Processing rows", unit="row") as pbar:
        for idx, row in df.iterrows():
            if idx >= target_rows:
                break

            row_id = row.get("id", idx)  # Use row index as fallback for ID

            if row_id in responses_df["name"].unique():
                pbar.update(1)
                continue
   
            text = row[author]
            if not isinstance(text, str) or pd.isna(text) or text.strip() == "":
                pbar.update(1)
                continue
            parsed = parser(text)

            for sentence_num, (sentence, context) in enumerate(zip(parsed["text"], parsed["context"])):
                try:
                    response = evaluator(sentence, context)
                    length = len(sentence.split())
                    context_length = len(context.split()) if context else 0

                    responses_df = pd.concat([
                        responses_df,
                        pd.DataFrame({
                            "num": [sentence_num + 1],
                            "length": [length],
                            "response": [response],
                            "context_length": [context_length],
                            "name": [row_id]
                        })
                    ], ignore_index=True)
                except Exception as e:
                    print(f"Error processing sentence: {sentence[:50]} -> {e}")

            # Save progress incrementally
            if checkpoint_path:
                with open(checkpoint_path, "w") as f:
                    f.write(str(idx + 1))
            responses_df.to_csv(save_path, index=False)
            pbar.update(1)

    print(f"Finished processing rows. Saved to {save_path}.")

In [5]:
model_names = ['meta-llama/Meta-Llama-3.1-8B-Instruct', 'microsoft/phi-2', 'tiiuae/falcon-7b', 'deepseek-ai/DeepSeek-R1-Distill-Qwen-7B']
datasets = ["src/wiki_dataset_clean.csv","src/news_dataset_clean.csv","src/abstracts_dataset_clean.csv"]
authors = ["human_text", "gpt", "Llama3.1_clean", "Falcon_clean", "R1_clean"]
target_rows=1500

for model_n in model_names:
    for dataset in datasets:
        for author in authors:
            print(f"Processing dataset: {dataset}, author: {author}, with LM: {model_n}")
            generate_responses(
                input_path=dataset,
                model_name=model_n,
                context_policy="none",
                author=author,
                target_rows=target_rows
            )

Processing dataset: src/wiki_dataset_clean.csv, author: human_text, with LM: meta-llama/Meta-Llama-3.1-8B-Instruct


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Processing rows: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1500/1500 [00:00<00:00, 6362.54row/s]


Finished processing rows. Saved to Responses/wiki_human_text_none_Meta-Llama-3.1-8B-Instruct.csv.
Processing dataset: src/wiki_dataset_clean.csv, author: gpt, with LM: meta-llama/Meta-Llama-3.1-8B-Instruct


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

...

Processing rows: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1500/1500 [10:46<00:00,  2.32row/s]


Finished processing rows. Saved to Responses/abstracts_Falcon_clean_none_DeepSeek-R1-Distill-Qwen-7B.csv.
Processing dataset: src/abstracts_dataset_clean.csv, author: R1_clean, with LM: deepseek-ai/DeepSeek-R1-Distill-Qwen-7B


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Processing rows: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1500/1500 [14:42<00:00,  1.70row/s]

Finished processing rows. Saved to Responses/abstracts_R1_clean_none_DeepSeek-R1-Distill-Qwen-7B.csv.
